In [31]:
# Model design
import agentpy as ap
import random

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

#Global variable
MOVES = [(1,0), (-1,0), (0,-1), (0,1)]

In [32]:
class MyModel(ap.Model):
    def setup(self):
        
        #Create agents
        trashAmount = 1000
        
        self.robot = ap.AgentList(self, self.p.agents)
        #self.garbage = ap.AgentList(self, self.p.dirt_density * (self.p.M * self.p.N)) 
        self.garbage = ap.AgentList(self, trashAmount) 
        
        #Create grid model
        self.area = ap.Grid(self, [self.p.M, self.p.N], track_empty=True)
        
        #Add the agents to the enviroment
        self.area.add_agents(self.robot, [(1,1)]*trashAmount)
        self.area.add_agents(self.garbage, random=True, empty=True)
        
        #Initiate dynamic variable for agents
        self.robot.condition = 0
        self.garbage.condition = 1
        
                
    def step(self):
        robots = self.robot
        for robot in robots:
            for neighbor in self.area.neighbors(robot):
                if neighbor.condition == 1:
                    neighbor.condition = 2
                    break;
                else:
                    self.area.move_by(robot, random.choice(MOVES))
                    
        #The experiment stops until there's no trash
        if len(self.garbage.select(self.garbage.condition == 1)) == 0:
            self.stop()
        
    def end(self):
         # Document a measure at the end of the simulation
        '''
        cleaned_trash = len(self.trash.select(self.trash.condition == 2))
        self.report('Percentage of cleaned trash', cleaned_trash / len(self.trash))
        '''
    
    

In [33]:
#Parametros

parameters = {
    'M': 50,
    'N': 40,
    'agents': 1,
    'dirt_density': 0.4,
    'setps': 100
}


In [34]:
def animation_plot(model, ax):
    attr_grid = model.area.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a forest fire\n"
                  f"Time-step: {model.t}, Trees left: "
                  f"{len(model.garbage.select(model.garbage.condition == 0))}")    

    
fig, ax = plt.subplots()
model = MyModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)


IPython.display.HTML(animation.to_jshtml())


RuntimeError: Set changed size during iteration